<a href="https://colab.research.google.com/github/SIDDHESH-8213/U-ASK-Winter-25/blob/main/U_ASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from rtree import index
from collections import defaultdict
from math import sqrt
import pandas as pd


In [ ]:
def load_data(filename):
    dataset = []
    with open(filename, "r") as file:
        for line in file:
            parts = line.strip().split()
            obj_id = int(parts[0])
            lat, lon = float(parts[1]), float(parts[2])
            keyword_count = int(parts[3])
            keywords = parts[4:4 + keyword_count * 2:2]
            dataset.append((obj_id, lat, lon, keyword_count, keywords))
    return dataset


data = load_data("/content/1.txt")

df = pd.DataFrame(data, columns=["Object ID", "Latitude", "Longitude", "Keyword Count", "Keywords"])

df.head()


,Object ID,Latitude,Longitude,Keyword Count,Keywords
0,100000,43.034281,-76.019289,4,"[red, 2, gourmet, burgers]"
1,100001,28.741791,-81.334263,1,[damn]
2,100002,38.529312,-121.431250,4,"[didn, score, give, put]"
3,100003,-6.194670,106.821030,1,[pinky]
4,100004,-30.099721,-51.159309,5,"[neon, night, wanna, party, lights]"


In [ ]:
spatial_idx = index.Index()

for obj_id, lat, lon, keywords_count, keywords in data:
    spatial_idx.insert(obj_id, (lon, lat, lon, lat))

#this is just to test
nearest = list(spatial_idx.nearest((-118.25, 34.0, -118.25, 34.0), 1))
print("Nearest location ID now:", nearest)


Nearest location ID: [136400]


In [ ]:
keyword_index = defaultdict(list)

for obj_id, lat, lon, keywords_count, keywords in data:
    for keyword in keywords:
        keyword_index[keyword].append(obj_id)

#again, just for testing
print("Places that serve burgers:", keyword_index["burgers"])


Places with 'burgers': [100000, 107529, 123991, 125641, 130720, 133192, 137851, 141867, 151424, 152299, 155560, 156863, 176072, 176077, 195706]
